In [34]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('Data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Data\sample_submission.csv
Data\test_features.csv
Data\train_drug.csv
Data\train_features.csv
Data\train_targets_nonscored.csv
Data\train_targets_scored.csv


In [35]:
visu_data = pd.read_csv('Data/train_features.csv')
visu_label = pd.read_csv('Data/train_targets_scored.csv')

In [36]:
visu_data.head()
#visu_label.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [37]:
from pydoc import locate
X_train = pd.DataFrame(pd.read_csv('Data/train_features.csv', nrows=5))
X_train.dtypes
types = [str(), str(), str(), str()]
for i in range(4, len(X_train.dtypes)):
    if X_train.dtypes[i].kind == 'f':
        types.append(float())


features = tf.data.experimental.CsvDataset('Data/train_features.csv', record_defaults=types, header=True)


In [38]:
y = pd.DataFrame(pd.read_csv('Data/train_targets_scored.csv', nrows=5))
types = ['']
for i in range(1, len(y.dtypes)):
    types.append(float())

targets = tf.data.experimental.CsvDataset('Data/train_targets_scored.csv', record_defaults=types, header=True)

In [39]:
dataset = tf.data.Dataset.zip((features, targets))

In [40]:
dataset_size = dataset.reduce(np.int64(0), lambda x, _:x+1).numpy()

In [41]:
train_dataset = dataset.take(0.7*dataset_size)
val_dataset = dataset.skip(0.7*dataset_size)
val_dataset = dataset.take(dataset_size - 0.7*dataset_size)

In [42]:
print(train_dataset.reduce(np.int64(0), lambda x, _:x+1).numpy())

16669


In [43]:
def _preprocess_line(features, targets):
    # Pack the result into a dictionary
    features = dict(zip(visu_data.columns, features))
    #print(features)
    features.pop('sig_id')

    targets = tf.stack(targets[1:])
    print(targets)
    return features, targets

train_dataset = train_dataset.map(_preprocess_line).batch(5)

Tensor("stack:0", shape=(206,), dtype=float32)


In [44]:
for feature_batch, label_batch in train_dataset.take(1):
    print('First 5 features:', list(feature_batch.keys())[:5])
    print('A batch of cp_types:', feature_batch['cp_type'].numpy())
    print('A batch of cp_times:', feature_batch['cp_time'].numpy())
    print('A batch of targets:', label_batch.numpy() )

First 5 features: ['cp_type', 'cp_time', 'cp_dose', 'g-0', 'g-1']
A batch of cp_types: [b'trt_cp' b'trt_cp' b'trt_cp' b'trt_cp' b'trt_cp']
A batch of cp_times: [b'24' b'72' b'48' b'48' b'72']
A batch of targets: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [47]:
import autosklearn as ask
#ask.regression.AutoSklearnRegressor() for regression tasks
model = ask.classification.AutoSklearnClassifier(ensemble_size=10, #size of the end ensemble (minimum is 1)
                                                 time_left_for_this_task=120, #the number of seconds the process runs for
                                                 per_run_time_limit=30) #maximum seconds allocated per model
model.fit(train_dataset, y_train) #begin fitting the search model
print(model.sprint_statistics()) #print statistics for the search
y_predictions = model.predict(X_test) #get predictions from the model

ModuleNotFoundError: No module named 'autosklearn'

In [48]:
from keras.models import Sequential
from keras.layers import Dense, Input
import tensorflow_addons as tfa

model2 = Sequential([
    Input(shape=(875,)),
    tf.keras.layers.Dense(2048, input_shape=(875,), activation='relu'),
    tf.keras.layers.Dense(206, activation='sigmoid')
])



Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow_addons'

In [49]:
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
for features, label in train_dataset.take(1):
    model2.fit(features, label, batch_size=32, epochs=10)


NameError: name 'model2' is not defined

In [51]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))
    for lab in label:
        print(lab)
show_batch(train_dataset)

cp_type             : [b'trt_cp' b'trt_cp' b'trt_cp' b'trt_cp' b'trt_cp']
cp_time             : [b'24' b'72' b'48' b'48' b'72']
cp_dose             : [b'D1' b'D1' b'D1' b'D1' b'D2']
g-0                 : [ 1.062   0.0743  0.628  -0.5138 -0.3254]
g-1                 : [ 0.5577  0.4087  0.5817 -0.2491 -0.4009]
g-2                 : [-0.2479  0.2991  1.554  -0.2656  0.97  ]
g-3                 : [-0.6208  0.0604 -0.0764  0.5288  0.6919]
g-4                 : [-0.1944  1.019  -0.0323  4.062   1.418 ]
g-5                 : [-1.012   0.5207  1.239  -0.8095 -0.8244]
g-6                 : [-1.022   0.2341  0.1715 -1.959  -0.28  ]
g-7                 : [-0.0326  0.3372  0.2155  0.1792 -0.1498]
g-8                 : [ 0.5548 -0.4047  0.0065 -0.1321 -0.8789]
g-9                 : [-0.0921  0.8507  1.23   -1.06    0.863 ]
g-10                : [ 1.183  -1.152  -0.4797 -0.8269 -0.2219]
g-11                : [ 0.153  -0.4201 -0.5631 -0.3584 -0.5121]
g-12                : [ 0.5574 -0.0958 -0.0366 -0.

g-578               : [-0.1023 -0.6138  0.4886  0.0803 -0.4986]
g-579               : [ 0.7075 -0.7623  0.6869 -0.0723  0.9745]
g-580               : [-0.2458  0.0863  0.504  -0.0759  1.492 ]
g-581               : [ 0.641   1.254  -0.4021  0.3699  0.3711]
g-582               : [ 0.745  -1.285   1.219   0.4403  0.0279]
g-583               : [-0.8205 -0.0172  0.7519  0.4006  0.6801]
g-584               : [-0.1733 -1.142   1.773  -0.3634 -0.1431]
g-585               : [-0.2182  0.532  -0.7739  0.769   1.041 ]
g-586               : [ 0.832  -0.2722  1.025  -0.7345  0.3855]
g-587               : [ 1.015   0.     -0.214   0.4005  0.7553]
g-588               : [-0.2779 -0.1406  0.0086  1.281   1.686 ]
g-589               : [-0.3271 -0.951  -0.4949  2.175   0.4932]
g-590               : [-1.245   0.9837  0.1489  0.6065 -0.2651]
g-591               : [ 0.6923 -0.2797 -1.889   1.898   1.183 ]
g-592               : [ 2.263  -1.194  -0.1239  0.848   0.9738]
g-593               : [ 0.505  -0.4952 -

In [52]:
for features, label in train_dataset.take(1):
    print(label)


tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(5, 206), dtype=float32)
